In [0]:
%run /eReport/PkgGeneral/PrcGeneral

In [0]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
class PrcRcdClasificacionInterna():
  
  def __init__(self, id_solicitud):
      self.id_solicitud = id_solicitud
      
      #Obteniendo parametros
      parametros = PrcGeneral(id_solicitud = self.id_solicitud).obtenerParametros()
      self.fecha_proceso = parametros['fecha_proceso']  
      self.fecha_proceso_str = self.fecha_proceso.strftime('%Y-%m-%d')
      
      #Fecha de actualizacion
      fecha_actualizacion = datetime.now()
      self.fecha_actualizacion_str = fecha_actualizacion.strftime("%Y-%m-%d %H:%M:%S")
      
      #Conexion
      connection = PrcGeneral().databaseConnect()
      self.connection = connection
      
      cursor = self.connection.cursor()
      self.cursor = cursor
      
  def ProcesaClasificacionInterna(self):

    #Obtenemos la cartera de clientes
    clasificacionInterna = """ SELECT A.ID CLIENTE_PRODUCTO, A.CLIENTE, A.DIAS_ATRASO, A.SITUACION_CREDITO, A.TIPO_CREDITO, B.CLASIFICACION_DEUDOR, A.CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO
    FROM CLIENTE_PRODUCTO A 
    JOIN CLASIFICACION_DEUDOR_DIAS_ATRASO B
    ON A.TIPO_CREDITO_SBS = B.TIPO_CREDITO_SBS
    AND A.DIAS_ATRASO BETWEEN B.DIAS_ATRASO_MINIMO AND ISNULL(B.DIAS_ATRASO_MAXIMO,DIAS_ATRASO+1)
    JOIN CLIENTE C ON C.ID = A.CLIENTE
    WHERE C.FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
    """
    sql_clasificacionInterna = pd.read_sql_query(clasificacionInterna, self.connection)
    df_clasificacionInterna = pd.DataFrame(sql_clasificacionInterna)
    
    #si es castigado 
    df_clasificacionInterna['CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO'] = np.where(df_clasificacionInterna['SITUACION_CREDITO']== 'C', 4, df_clasificacionInterna['CLASIFICACION_DEUDOR'])
    
    #si es refinanciado 
    df_clasificacionInterna['CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO_FLAG'] = np.where(df_clasificacionInterna['TIPO_CREDITO']==2, 9, np.where(df_clasificacionInterna['CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO'] < df_clasificacionInterna['CLASIFICACION_DEUDOR'], 9, df_clasificacionInterna['CLASIFICACION_DEUDOR']))
  
    df_clasificacionInterna['CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO'] = np.where(df_clasificacionInterna['CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO_FLAG']==9,df_clasificacionInterna['CLASIFICACION_DEUDOR'],df_clasificacionInterna['CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO'])
    
    df_clasificacionInterna_cp = df_clasificacionInterna[['CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO','CLIENTE_PRODUCTO']]
    df_clasificacionInterna_c = df_clasificacionInterna[['CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO','CLASIFICACION_DEUDOR','CLIENTE']]
 
    #actualizamos la clasificacion interna de cliente producto y Cliente     
    actualizaClasificacionInterna_cp = "UPDATE [DBO].CLIENTE_PRODUCTO SET CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO= ? where id =?" #OBS: quitar new 
    self.cursor.fast_executemany = True
    self.cursor.executemany(actualizaClasificacionInterna_cp, df_clasificacionInterna_cp.values.tolist())
    self.connection.commit()    
    
    actualizaClasificacionInterna_c = "UPDATE [DBO].CLIENTE SET CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO= ? , CLASIFICACION_DEUDOR= ? where id =?" 
    self.cursor.fast_executemany = True
    self.cursor.executemany(actualizaClasificacionInterna_c, df_clasificacionInterna_c.values.tolist())
    self.connection.commit()   
    self.cursor.close()
    
objeto = PrcRcdClasificacionInterna(4)
objeto.ProcesaClasificacionInterna()